<a href="https://colab.research.google.com/github/utautage/calculator/blob/main/%E9%9B%BB%E5%8D%93%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実験2（High and Low）

In [ ]:
import random
answer = int(random.random() * 100)
while True:
    guess = int(input("私の考えた0〜99の数はな〜んだ？"))
    if answer == guess:
        print("Bingo!!")
        break
    elif answer < guess:
        print("too High")
    else:
        print("too Low")

私の考えた0〜99の数はな〜んだ？50
too Low
私の考えた0〜99の数はな〜んだ？75
too High
私の考えた0〜99の数はな〜んだ？63
too High
私の考えた0〜99の数はな〜んだ？57
too High
私の考えた0〜99の数はな〜んだ？54
too Low
私の考えた0〜99の数はな〜んだ？55
too Low
私の考えた0〜99の数はな〜んだ？56
Bingo!!


# 実験3〜6（電卓プログラミング）

## 実験3（数の認識）

入力`string`には、EndOfLineとして改行文字`\n`を追加しておく。これが無いと、数字の次の文字を読んでエラーになる。文字列の長さをチェックする方法もあるが、そちらは実装が面倒なので、こちらの方が楽ではある。

In [ ]:
def extractNumber(string, index):
    result = ""
    if string[index] == "-":
        result += string[index]
        index += 1
    if string[index] == ".":
        result += string[index]
        index += 1
        while str.isdigit(string[index]):
            result += string[index]
            index += 1
    else:
        while str.isdigit(string[index]):
            result += string[index]
            index += 1
        if string[index] == ".":
            result += string[index]
            index += 1
        while str.isdigit(string[index]):
            result += string[index]
            index += 1
    return float(result), index

string = input("実数を入力してください")
string += "\n"
result = extractNumber(string, 0)
print(result)
print(result[0])

実数を入力してください1
(1.0, 1)
1.0


## 実験4（加減算）


In [ ]:
def calculateExpression(string, index):
    term, index = calculateTerm(string, index)
    match string[index]:
        case "+":
            expression, index = calculateExpression(string, index + 1)
            term += expression
        case "-":
            expression, index = calculateExpression(string, index + 1)
            term -= expression
    return term, index

string = input("実数の式を入力してください")
string += "\n"
result = calculateExpression(string, 0)
print(result[0])

実数の式を入力してください1


NameError: name 'calculateTerm' is not defined

## 実験5（四則演算）

In [ ]:
def calculateTerm(string, index):
    factor, index = calculateFactor(string, index)
    match string[index]:
        case "*":
            term, index = calculateTerm(string, index + 1)
            factor *= term
        case "/":
            term, index = calculateTerm(string, index + 1)
            factor /= term
    return factor, index

string = input("実数の式を入力してください")
string += "\n"
result = calculateExpression(string, 0)
print(result[0])

実数の式を入力してください1


NameError: name 'extractNumber' is not defined

## 実験6（括弧を使う）

In [ ]:
def calculateFactor(string, index):
    if string[index] == "(":
        expression, index = calculateExpression(string, index + 1)
        return expression, index + 1 # この時点でポインタは閉じ括弧を指しているはずなので、ポインタを1文字進める
    else:
        return extractNumber(string, index)

string = input("実数の式を入力してください")
string += "\n"
result = calculateExpression(string, 0)
print(result[0])

実数の式を入力してください1
1.0


# 考察

## 三角関数や、その他の算術関数を追加するにはどうしたらよいだろうか？

関数を実装するために、因子の構文をBNF記法で次のように変更する。

```
<因子> ::= <数> | "(" <式> ")" | <関数>
<関数> ::= <識別子> "(" (<式> ("," <式>)*)? ")"
<識別子> ::= (<英字> | <アンダースコア>) (<英字> | <数字> | <アンダースコア>)*
<英字> ::= "a" | "b" | "c" | "d" | "e" | "f" | "g" | "h" | "i" | "j" | "k" | "l" | "m" | "n" | "o" | "p" | "q" | "r" | "s" | "t" | "u" | "v" | "w" | "x" | "y" | "z"
<アンダースコア> ::= "_"
```

関数の定義を分かりやすく書くと`関数名(引数1[,引数2...])`となる。英字の大小は区別せず、計算の過程で大文字は小文字に変換されるとする。

そこで、`calculateFactor`の仕様を次のように変更する。

> 読み込んだ文字が…
> - 数字なら、単なる数である。`extractNumber`で数を検出する。
> - 開き括弧`(`なら、次の文字から式が現れる。ポインタを1文字進め、`calculateExpression`で式を評価する。
> - 英字かアンダースコアなら、識別子である。
>   1. 英字・数字・アンダースコア以外の文字が出るまで、ポインタを進めながら文字を読んでいく。
>   1. 識別子の後には開き括弧`(`が現れる（そうでなければ変数だが、その実装は時間がなくてできなかった）。ポインタを1文字進める。
>   1. 引数を読み取る。
>   1. 検出した関数名や引数の値から適当な値を返す。

引数を読み取る処理は次のように実装する。

> 読み込んだ文字が…
> - 閉じ括弧`)`なら、これ以上先に引数は無い。ポインタを1文字進め、処理を打ち切る。
> - 閉じ括弧でなければ、引数として式が現れる。
>   1. `calculateExpression`で式を評価する。
>   1. 式の後にはカンマ`,`か閉じ括弧が現れる。ポインタを1文字進める。
>   1. 処理全体をもう一度繰り返す。

In [ ]:
import math

def calculateFactor(string, index):
    if str.isdigit(string[index]):
        return extractNumber(string, index)
    elif string[index] == "(":
        expression, index = calculateExpression(string, index + 1)
        return expression, index + 1 # この時点でポインタは閉じ括弧を指しているはずなので、ポインタを1文字進める
    elif str.isalpha(string[index]) or string[index] == "_":
        identifier = string[index]
        index += 1
        while str.isalnum(string[index]) or str.isdigit(string[index]) or string[index] == "_":
            identifier += string[index]
            index += 1
        identifier = identifier.lower()
        index += 1 # この時点でポインタは開き括弧を指しているはずなので、ポインタを1文字進める
        arguments = []
        while True:
            if string[index] == ")":
                index += 1
                break
            argument, index = calculateExpression(string, index)
            arguments.append(argument)
        match identifier:
            case "sin":
                return math.sin(arguments[0]), index
            case "cos":
                return math.cos(arguments[0]), index
            case "pi":
                return math.pi, index

string = input("実数の式を入力してください")
string += "\n"
result = calculateExpression(string, 0)
print(result[0])

実数の式を入力してくださいsin(pi()/2)
1.0


## 変数を使えるようにするにはどうしたらよいだろうか？

変数の代入は、`<変数名> = <式>`で行うことにする。それに伴い、通常の演算は、`= <式>`で行うことにする。

`=`の右側に識別子が単体で（後ろに`(`が付かない→関数ではない）出てきたときは、保存されている値に置き換えて計算する。

## 式に余分な空白があったらどうしようか？

現状のプログラムでは、式に空白文字を含むケースが想定されていない。この場合、エラーが発生して正常に計算することができない。

逆に言えば、式の空白文字を削除する、つまり空文字列に置き換えることで、正常に計算できるようになるはずである。

In [ ]:
string = input("実数の式を入力してください")
string += "\n"
string = string.replace(" ", "") # 追加。スペースを空文字列に置き換え
result = calculateExpression(string, 0)
print(result[0])

実数の式を入力してください5 * (1 + 2) - 6 / 3
13.0


## 文法エラーを見つけたらどうしようか？

エラーを投げる。

具体的には、次のような場合を検出したら、エラーメッセージを出力して処理を中断する。
- 因子の終わりに`)`がない。
- 演算に使われない文字（英数字・アンダースコア・括弧・四則演算子以外）がある。
- まだ代入されていない変数（識別子）が計算式に登場している。